In [131]:
import platform
from collections import Counter

import numpy as np

# tokenizer import
from konlpy.tag import Okt, Komoran, Hannanum, Kkma

#운영체제에 따라 mecab설치 방법이 다름.
if platform.system() == "Windows":
    try:
        from eunjeon import Mecab
    except:
        print("please install eunjeon module")
else:  # Ubuntu일 경우
    from konlpy.tag import Mecab

from typing import List, Callable, Union, Any, TypeVar, Tuple, Dict

from functools import partial
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
import gensim
from gensim import corpora
import kss



In [132]:
with open("./sents.txt", "r") as f:
    sents = f.read().split("\n")
print(sents)

['오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다', '서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다', '경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다', '이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다', '성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다', '이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다', '5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다', '용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기', '신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다', '김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 

In [133]:
news1 = """ 국내 신종 코로나바이러스 감염증(코로나19) 신규 확진자가 하루만에 600명대로 올라섰다.전국적으로 크고 작은 일상 감염이 잇따르는 상황에서 해외유입 변이 바이러스도 빠르게 확산되고 있어 당국이 촉각을 세우고 있다. 정부는 오는 21일 다음주부터 적용할 사회적 거리두기 조정안을 발표할 예정이다.19일 중앙방역대책본부(중대본)에 따르면 이날 0시 기준 코로나19 신규 확진자는 654명으로 전날(528명)보다 126명 늘었다.누적 확진자는 13만3471명이다.확진자 수는 보통 주말·휴일 검사건수 감소 영향으로 인해 주 초반에는 비교적 적게 나오다가 중반부터 증가하는 경향을 보인다.이날 신규 확진자의 감염경로는 지역발생이 637명, 해외유입이 17명이다.지역별로는 서울 245명, 경기 159명, 인천 23명 등 수도권이 427명(67%)이다.주요 집단발병 사례를 보면 노래연습장, 유흥업소 등 다중이용시설을 고리로 새로운 감염이 확인됐다.수도권에서는 서울 강동구 노래연습장과 관련해 41명, 서울 노원구 고시원에서 11명, 경기 성남시 일가족-지인 사례에서 19명이 각각 양성 판정을 받았다.앞서 발생한 인천국제공항 검역소 집단감염과 관련해선 8명이 인도 변이에 감염된 것으로 확인됐다. 역학적 관련성이 있는 나머지 7명까지 포함하면 15명 전원이 인도 변이 감염자인 셈이다.최근 코로나19 발생 양상을 보면 전국에서 집단감염이 잇따르면서 '4차 유행'이 지속하고 있다.지난 13일부터 이날까지 최근 1주일간 하루 평균 신규 확진자는 약 651명꼴이다. 이중 '사회적 거리두기' 단계 조정의 핵심 지표인 일평균 지역발생 확진자는 약 628명으로, 여전히 2.5단계(전국 400∼500명 이상 등) 범위다.사망자는 전날보다 8명 늘어 누적 1912명이 됐다. 국내 평균 치명률은 1.43%다.전날 하루 선별진료소를 통한 검사 건수는 3만3640건으로, 직전일(4만1704건)보다 19% 적다.검사건수 대비 확진자를 계산한 양성률은 1.94%(3만3640명 중 654명)로, 직전일 1.27%(4만1704명 중 528명)보다 상승했다.정부는 오는 21일 다음주부터 3주간 적용할 사회적 거리두기 조정안을 발표한다. 현재 수도권에서는 2단계, 비수도권에서는 1.5단계의 거리두기 조처가 이뤄지고 있다."""

In [134]:
news2 = """경북 영천에서 17일 하루 만에 신종 코로나바이러스 감염증(코로나19) 확진자가 5명이 발생, N차 감염이 우려되는 가운데 비상이 걸렸다. 지난 7일 80번 확진자 이후 열흘 만에 지역 누적 확진자는 85명으로 늘어났다.영천시에 따르면 경산에 주소를 둔 공부방 교사가 확진됐다는 소식을 접한 A여고 한 학생이 지난 16일 보건소 검체 검사 결과 17일 양성판정을 받았다.또 확진자 교사가 운영 중인 공부방에서 과외 학습을 받고 있는 B여고 학생이 17일 오후 8시 확진 판정을 받은 가운데 A여고 학생 부모와 B여고 학생 엄마 등 총 5명이 코로나 판정을 받았다.수업을 받은 나머지 학생들은 모두 음성 판정을 받은 것으로 알려졌다.이날 A여고 측은 학생, 교직원 등 140여 명이 보건소에서 검체를 실시했으며 뒤늦게 알려진 B여고는 18일 학생과 교직원들이 검사를 받을 예정이다.이에 보건소는 코로나 확진자들을 자가 격리하고 이들의 이동 동선 파악에 집중하는 가운데 학교 등의 방역소독을 철저히 하고 있다."""


In [135]:
news3 =""" 브라질, 인도발 코로나 변이 상륙할라…긴장 속 모니터링 강화 (상파울루=연합뉴스) 김재순 특파원 = 브라질 보건당국이 인도발 신종 코로나바이러스 감염증(코로나19) 변이 바이러스 상륙 가능성에 긴장하고 있다.18일(현지시간) 브라질 언론에 따르면 상파울루시 당국은 인도발 변이 바이러스가 브라질에서도 나타날 가능성이 크다는 지적에 따라 상파울루주 정부 산하 부탄탕연구소와 함께 모니터링을 강화하기로 했다고 밝혔다.상파울루시는 브라질에서 코로나19 확진자와 사망자가 가장 먼저 나온 곳으로, 인근 과룰류스 국제공항을 통해 외국인 입국이 대규모로 이뤄지고 있다.상파울루시 관계자는 이날 기자회견을 통해 "인도발 변이 바이러스 상륙을 막기 위한 사전 조치로 부탄탕연구소와 협력해 외국인에 대한 철저한 추적 관찰이 이뤄질 것"이라고 말했다."""


In [136]:
news4 = """ 코로나 방역 모범국으로 평가받던 대만, 베트남, 싱가포르에서 최근 코로나 확진자가 크게 늘면서 방역에 비상이 걸렸다. 결국 백신 접종이 이뤄지지 않으면 코로나 확산세를 억제하기는 어렵다는 지적이 나온다. 미 블룸버그통신은 대만과 싱가포르가 자국내 방역 단계를 빠르게 격상시키고 있다고 16일 보도했다. 대만 정부는 이번 주말에 시민들이 집에서 머무르도록 권고하고, 실내 모임은 최대 5명, 야외 모임은 10명까지 제한했다. 국제 통계사이트 월드오미터 기준 대만에서 16일 보고된 코로나 신규 확진자는 207명으로 최고치를 기록했다.싱가포르 역시 식당 내에서의 식사를 금지하고 재택근무를 의무화했으며, 사적 모임은 최대 2인까지로 제한했다. 싱가포르 정부가 이런 봉쇄 정책을 실시한 것은 지난해 4~5월 이후 1년 만이다. 16일 보고된 싱가포르 신규 확진자는 49명이지만, 5월 초부터 집단 감염이 발생하자 경계 수위를 높였다. 또 최근 21일간 대만 여행 이력이 있는 외국인의 입국을 금지했다.대만과 싱가포르는 지난 12월 양국 간의 여행 제한 조치를 완화했었다. 양국에서 서로 체류하다 온 이들에 대해 자가격리 절차를 면제해 주기로 한 것이다. 하지만 자국 내 코로나 확산세가 심해지면서 여행객들에 대한 빗장도 다시 걸어 잠갔다.베트남은 이날 월드오미터 기준 127명의 신규 확진자가 보고됐다. 그 전날에는 165명의 확진자가 보고됐는데, 이는 지난해 1월 베트남에서 코로나가 처음 발병한 이래 최고치다. 북부 바짱주의 꽝차우 산업단지에서 집단감염이 일어나면서 확진자 수가 치솟았다.블룸버그는 이런 현상에 대해 “지역 사회에 대한 백신 접종 없이 바이러스로부터 안전한 환경을 만드는 게 얼마나 어려운지를 보여주는 사례”라고 했다. 블룸버그에 따르면 대만의 백신 접종률은 1% 미만이고, 베트남 역시 백신 접종률은 인구의 1% 수준이다. 싱가포르는 발빠르게 백신 확보에 나선 나라 중 하나이지만, 지금까지 한번이라도 백신을 맞은 국민은 전체 인구의 3분의 1 정도다."""


In [137]:
stop = "직전,아,휴,아이구,아이쿠,아이고,어,나,우리,저희,따라,의해,을,를,에,의,가,으로,로,에게,뿐이다,의거하여,근거하여,입각하여,기준으로,예하면,예를 들면,예를 들자면,저,소인,소생,저희,지말고,하지마,하지마라,다른,물론,또한,그리고,비길수 없다,해서는 안된다,뿐만 아니라,만이 아니다,만은 아니다,막론하고,관계없이,그치지 않다,그러나,그런데,하지만,든간에,논하지 않다,따지지 않다,설사,비록,더라도,아니면,만 못하다,하는 편이 낫다,불문하고,향하여,향해서,향하다,쪽으로,틈타,이용하여,타다,오르다,제외하고,이 외에,이 밖에,하여야,비로소,한다면 몰라도,외에도,이곳,여기,부터,기점으로,따라서,할 생각이다,하려고하다,이리하여,그리하여,그렇게 함으로써,하지만,일때,할때,앞에서,중에서,보는데서,으로써,로써,까지,해야한다,일것이다,반드시,할줄알다,할수있다,할수있어,임에 틀림없다,한다면,등,등등,제,겨우,단지,다만,할뿐,딩동,댕그,대해서,대하여,대하면,훨씬,얼마나,얼마만큼,얼마큼,남짓,여,얼마간,약간,다소,좀,조금,다수,몇,얼마,지만,하물며,또한,그러나,그렇지만,하지만,이외에도,대해 말하자면,뿐이다,다음에,반대로,반대로 말하자면,이와 반대로,바꾸어서 말하면,바꾸어서 한다면,만약,그렇지않으면,까악,툭,딱,삐걱거리다,보드득,비걱거리다,꽈당,응당,해야한다,에 가서,각,각각,여러분,각종,각자,제각기,하도록하다,와,과,그러므로,그래서,고로,한 까닭에,하기 때문에,거니와,이지만,대하여,관하여,관한,과연,실로,아니나다를가,생각한대로,진짜로,한적이있다,하곤하였다,하,하하,허허,아하,거바,와,오,왜,어째서,무엇때문에,어찌,하겠는가,무슨,어디,어느곳,더군다나,하물며,더욱이는,어느때,언제,야,이봐,어이,여보시오,흐흐,흥,휴,헉헉,헐떡헐떡,영차,여차,어기여차,끙끙,아야,앗,아야,콸콸,졸졸,좍좍,뚝뚝,주룩주룩,솨,우르르,그래도,또,그리고,바꾸어말하면,바꾸어말하자면,혹은,혹시,답다,및,그에 따르는,때가 되어,즉,지든지,설령,가령,하더라도,할지라도,일지라도,지든지,몇,거의,하마터면,인젠,이젠,된바에야,된이상,만큼,어찌됏든,그위에,게다가,점에서 보아,비추어 보아,고려하면,하게될것이다,일것이다,비교적,좀,보다더,비하면,시키다,하게하다,할만하다,의해서,연이서,이어서,잇따라,뒤따라,뒤이어,결국,의지하여,기대여,통하여,자마자,더욱더,불구하고,얼마든지,마음대로,주저하지 않고,곧,즉시,바로,당장,하자마자,밖에 안된다,하면된다,그래,그렇지,요컨대,다시 말하자면,바꿔 말하면,즉,구체적으로,말하자면,시작하여,시초에,이상,허,헉,허걱,바와같이,해도좋다,해도된다,게다가,더구나,하물며,와르르,팍,퍽,펄렁,동안,이래,하고있었다,이었다,에서,로부터,까지,예하면,했어요,해요,함께,같이,더불어,마저,마저도,양자,모두,습니다,가까스로,하려고하다,즈음하여,다른,다른 방면으로,해봐요,습니까,했어요,말할것도 없고,무릎쓰고,개의치않고,하는것만 못하다,하는것이 낫다,매,매번,들,모,어느것,어느,로써,갖고말하자면,어디,어느쪽,어느것,어느해,어느 년도,라 해도,언젠가,어떤것,어느것,저기,저쪽,저것,그때,그럼,그러면,요만한걸,그래,그때,저것만큼,그저,이르기까지,할 줄 안다,할 힘이 있다,너,너희,당신,어찌,설마,차라리,할지언정,할지라도,할망정,할지언정,구토하다,게우다,토하다,메쓰겁다,옆사람,퉤,쳇,의거하여,근거하여,의해,따라,힘입어,그,다음,버금,두번째로,기타,첫번째로,나머지는,그중에서,견지에서,형식으로 쓰여,입장에서,위해서단지,의해되다,하도록시키다,뿐만아니라,반대로,전후,전자,앞의것,잠시,잠깐,하면서,그렇지만,다음에,그러한즉,그런즉,남들,아무거나,어찌하든지,같다,비슷하다,예컨대,이럴정도로,어떻게,만약,만일,위에서 서술한바와같이,인 듯하다,하지 않는다면,만약에,무엇,무슨,어느,어떤,아래윗,조차,한데,그럼에도 불구하고,여전히,심지어,까지도,조차도,하지 않도록,않기 위하여,때,시각,무렵,시간,동안,어때,어떠한,하여금,네,예,우선,누구,누가 알겠는가,아무도,줄은모른다,줄은 몰랏다,하는 김에,겸사겸사,하는바,그런 까닭에,한 이유는,그러니,그러니까,때문에,그,너희,그들,너희들,타인,것,것들,너,위하여,공동으로,동시에,하기 위하여,어찌하여,무엇때문에,붕붕,윙윙,나,우리,엉엉,휘익,윙윙,오호,아하,어쨋든,만 못하다,하기보다는,차라리,하는 편이 낫다,흐흐,놀라다,상대적으로 말하자면,마치,아니라면,쉿,그렇지 않으면,그렇지 않다면,안 그러면,아니었다면,하든지,아니면,이라면,좋아,알았어,하는것도,그만이다,어쩔수 없다,하나,일,일반적으로,일단,한켠으로는,오자마자,이렇게되면,이와같다면,전부,한마디,한항목,근거로,하기에,아울러,하지 않도록,않기 위해서,이르기까지,이 되다,로 인하여,까닭으로,이유만으로,이로 인하여,그래서,이 때문에,그러므로,그런 까닭에,알 수 있다,결론을 낼 수 있다,으로 인하여,있다,어떤것,관계가 있다,관련이 있다,연관되다,어떤것들,에 대해,이리하여,그리하여,여부,하기보다는,하느니,하면 할수록,운운,이러이러하다,하구나,하도다,다시말하면,다음으로,에 있다,에 달려 있다,우리,우리들,오히려,하기는한데,어떻게,어떻해,어찌됏어,어때,어째서,본대로,자,이,이쪽,여기,이것,이번,이렇게말하자면,이런,이러한,이와 같은,요만큼,요만한 것,얼마 안 되는 것,이만큼,이 정도의,이렇게 많은 것,이와 같다,이때,이렇구나,것과 같이,끼익,삐걱,따위,와 같은 사람들,부류의 사람들,왜냐하면,중의하나,오직,오로지,에 한하다,하기만 하면,도착하다,까지 미치다,도달하다,정도에 이르다,할 지경이다,결과에 이르다,관해서는,여러분,하고 있다,한 후,혼자,자기,자기집,자신,우에 종합한것과같이,총적으로 보면,총적으로 말하면,총적으로,대로 하다,으로서,참,그만이다,할 따름이다,쿵,탕탕,쾅쾅,둥둥,봐,봐라,아이야,아니,와아,응,아이,참나,년,월,일,령,영,일,이,삼,사,오,육,륙,칠,팔,구,이천육,이천칠,이천팔,이천구,하나,둘,셋,넷,다섯,여섯,일곱,여덟,아홉,령,영 "
print(type(stop))

<class 'str'>


In [138]:
news = [news1,news2,news3,news4]
sents=news

In [139]:
print(news)
print(len(news))

#Textrank 용
mecab = Mecab()
stop_words = stop.split(',')
text_list = kss.split_sentences(news1+news2+news3+news4)
tl = []
for i in text_list :
    sentences = mecab.nouns(i)
    result2 = []
    #print('sentences')
    #print(sentences)
    for word in sentences :
        #print(word)
        if word not in stop_words:
            result2.append(word)
    #print('result')
    #print(result)
    tl.append(result2)

print(len(tl))
print(tl, '\n')
    

[" 국내 신종 코로나바이러스 감염증(코로나19) 신규 확진자가 하루만에 600명대로 올라섰다.전국적으로 크고 작은 일상 감염이 잇따르는 상황에서 해외유입 변이 바이러스도 빠르게 확산되고 있어 당국이 촉각을 세우고 있다. 정부는 오는 21일 다음주부터 적용할 사회적 거리두기 조정안을 발표할 예정이다.19일 중앙방역대책본부(중대본)에 따르면 이날 0시 기준 코로나19 신규 확진자는 654명으로 전날(528명)보다 126명 늘었다.누적 확진자는 13만3471명이다.확진자 수는 보통 주말·휴일 검사건수 감소 영향으로 인해 주 초반에는 비교적 적게 나오다가 중반부터 증가하는 경향을 보인다.이날 신규 확진자의 감염경로는 지역발생이 637명, 해외유입이 17명이다.지역별로는 서울 245명, 경기 159명, 인천 23명 등 수도권이 427명(67%)이다.주요 집단발병 사례를 보면 노래연습장, 유흥업소 등 다중이용시설을 고리로 새로운 감염이 확인됐다.수도권에서는 서울 강동구 노래연습장과 관련해 41명, 서울 노원구 고시원에서 11명, 경기 성남시 일가족-지인 사례에서 19명이 각각 양성 판정을 받았다.앞서 발생한 인천국제공항 검역소 집단감염과 관련해선 8명이 인도 변이에 감염된 것으로 확인됐다. 역학적 관련성이 있는 나머지 7명까지 포함하면 15명 전원이 인도 변이 감염자인 셈이다.최근 코로나19 발생 양상을 보면 전국에서 집단감염이 잇따르면서 '4차 유행'이 지속하고 있다.지난 13일부터 이날까지 최근 1주일간 하루 평균 신규 확진자는 약 651명꼴이다. 이중 '사회적 거리두기' 단계 조정의 핵심 지표인 일평균 지역발생 확진자는 약 628명으로, 여전히 2.5단계(전국 400∼500명 이상 등) 범위다.사망자는 전날보다 8명 늘어 누적 1912명이 됐다. 국내 평균 치명률은 1.43%다.전날 하루 선별진료소를 통한 검사 건수는 3만3640건으로, 직전일(4만1704건)보다 19% 적다.검사건수 대비 확진자를 계산한 양성률은 1.94%(3만3640명 중 654명)로, 직전일

In [140]:
bigram = gensim.models.Phrases(tl, min_count = 2, threshold = 0.000000000000000000001)
print(bigram)
#trigram = gensim.models.Phrases(bigram[j],min_count = 1, threshold = 0.000000000000000000001)

bigram_model = gensim.models.phrases.Phraser(bigram)
#trigram_model = gensim.models.phrases.Phraser(trigram)
#quadgram_model = gensim.models.phrases.Phraser(quadgrams)

bigram_document = [bigram_model[nouns] for nouns in tl]
#trigram_document = [trigram_model[nouns] for nouns in j]

print(tl[0])
print(bigram_document[0])
print(tl[1])
print(bigram_document[1])

print()
print(bigram_document)
#print(trigram_document)
print()

Phrases<707 vocab, min_count=2, threshold=1e-21, max_vocab_size=40000000>
['국내', '신종', '코로나', '바이러스', '감염증', '코로나', '신규', '확진', '하루', '만', '명']
['국내', '신종_코로나', '바이러스_감염증', '코로나', '신규_확진', '하루', '만_명']
['전국', '일상', '감염', '상황', '해외', '유입', '변이', '바이러스', '확산', '당국', '촉각']
['전국', '일상', '감염', '상황', '해외', '유입', '변이_바이러스', '확산', '당국', '촉각']

[['국내', '신종_코로나', '바이러스_감염증', '코로나', '신규_확진', '하루', '만_명'], ['전국', '일상', '감염', '상황', '해외', '유입', '변이_바이러스', '확산', '당국', '촉각'], ['정부', '주', '적용', '사회_거리', '두기', '조정안', '발표', '예정'], ['중앙', '방역', '대책', '본부', '중대', '이날', '시', '기준', '코로나', '신규_확진', '명', '전날_명', '명'], ['누적', '확진', '만_명'], ['확진', '수', '보통', '주말', '휴일', '검사_건수', '감소', '영향', '초반', '비교', '중반', '증가', '경향'], ['이날', '신규_확진', '감염', '경로', '지역', '발생', '명', '해외', '유입', '명'], ['지역', '서울', '명', '경기', '명', '인천', '명', '수도', '명'], ['주요', '집단', '발병', '사례', '노래', '연습장', '유흥업소', '다중', '이용', '시설', '고리', '감염', '확인'], ['수도권', '서울', '강동구', '노래', '연습장', '관련', '명', '서울', '노원구', '고시원', '명', '경기', '가족', '지인', '사례', '명',

In [141]:
def vectorize_sents(sents: List[str], stopwords=None, min_count=2, tokenizer="mecab", noun=True):
   

    vectorizer = CountVectorizer(tokenizer=lambda x: x,lowercase=False)

    vec = vectorizer.fit_transform(bigram_document)
    print(vec)
    vocab_idx = vectorizer.vocabulary_
    print(vocab_idx)
    idx_vocab = {idx: vocab for vocab, idx in vocab_idx.items()}
    print(idx_vocab)
    
    return vec, vocab_idx, idx_vocab

In [142]:
def word_similarity_matrix(x, min_sim=0.3):
    sim_mat = 1 - pairwise_distances(x.T, metric="cosine")
    sim_mat[np.where(sim_mat <= min_sim)] = 0

    return sim_mat

In [143]:
def word_graph(
    sents: List[str],
    min_count=2,
    min_sim=0.3,
    tokenizer="mecab",
    noun=True,
    stopwords: List[str] = ["연합뉴스", "그리고", "기자"],
):

    mat, vocab_idx, idx_vocab = vectorize_sents(
        sents, stopwords, min_count=min_count, tokenizer=tokenizer, noun=noun
    )

    mat = word_similarity_matrix(mat, min_sim=min_sim)

    return mat, vocab_idx, idx_vocab

In [144]:
def pagerank(x: np.ndarray, df=0.85, max_iter=50, method="iterative"):

    assert 0 < df < 1

    A = normalize(mat, axis=0, norm="l1")
    N = np.ones(A.shape[0]) / A.shape[0]

    if method == "iterative":
        R = np.ones(A.shape[0])
        for _ in range(max_iter):
            R = df * np.matmul(A, R) + (1 - df) * N
    elif method == "algebraic":
        R = np.linalg.inv((I - df * A))
        R = np.matmul(R, (1 - df) * N)

    return R

In [145]:
mat,vocab_idx, idx_vocab = word_graph(sents)
R = pagerank(mat,method ="iterative")
topk = 10
idxs = R.argsort()[-topk:]
keywords = [(idx, R[idx], idx_vocab[idx]) for idx in reversed(idxs)]
keyword = [(R[idx]) for idx in reversed(idxs)]

AttributeError: 'list' object has no attribute 'lower'

In [146]:
keywords

[(33, 0.022927395999148576, '이날'),
 (48, 0.02234111218256318, '코로나'),
 (54, 0.021844150916388954, '확진'),
 (39, 0.02156335352608398, '정부'),
 (42, 0.020912371095318485, '지역'),
 (15, 0.0203363937910987, '발생'),
 (0, 0.02026975697232176, '감염'),
 (44, 0.020128252109039495, '집단'),
 (23, 0.02009101181436368, '신규'),
 (24, 0.02009101181436368, '신규 확진')]